# [Day 7: No Space Left On Device](https://adventofcode.com/2022/day/7)

In [1]:
example = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

Parse commands and output.

In [2]:
def parse(buffer):
    """Generates commands and their output."""
    command = []
    for line in buffer.splitlines():
        if line.startswith('$'):
            if command:
                yield command
            # Split command line into command and arguments.
            command = [line.split()[1:]]
        else: 
            command.append(line.split())
    yield command
    
list(parse(example))

[[['cd', '/']],
 [['ls'],
  ['dir', 'a'],
  ['14848514', 'b.txt'],
  ['8504156', 'c.dat'],
  ['dir', 'd']],
 [['cd', 'a']],
 [['ls'], ['dir', 'e'], ['29116', 'f'], ['2557', 'g'], ['62596', 'h.lst']],
 [['cd', 'e']],
 [['ls'], ['584', 'i']],
 [['cd', '..']],
 [['cd', '..']],
 [['cd', 'd']],
 [['ls'],
  ['4060174', 'j'],
  ['8033020', 'd.log'],
  ['5626152', 'd.ext'],
  ['7214296', 'k']]]

Build file system.

In [3]:
import pathlib

def build_fs(commands):
    "Returns filesystem constructed from commands."
    fs = {}
    cwd = pathlib.PurePath()
    for (command, *args), *output in commands:
        match command:
            # cd changes the state of the current directory.
            case 'cd':
                dir = args[0]
                if dir == '..':
                    cwd = cwd.parent
                else:
                    cwd = cwd.joinpath(args[0])
            # The output of ls describes the contents of the current directory.
            case 'ls':
                # Convert size to int if not a directory.
                fs[cwd] = {(int(size), name) for size, name in output if size != 'dir'}
    return fs

build_fs(parse(example))

{PurePosixPath('/'): {(8504156, 'c.dat'), (14848514, 'b.txt')},
 PurePosixPath('/a'): {(2557, 'g'), (29116, 'f'), (62596, 'h.lst')},
 PurePosixPath('/a/e'): {(584, 'i')},
 PurePosixPath('/d'): {(4060174, 'j'),
  (5626152, 'd.ext'),
  (7214296, 'k'),
  (8033020, 'd.log')}}

Calculate total directory sizes.

In [4]:
def total(fs):
    """Returns total sizes of directories of filesystem."""
    size = {}
    for path, contents in fs.items():
        # Sum directory size.
        total = sum(size for size, name in contents)
        size[path] = size.get(path, 0) + total
    
        # Add total to parent directorys too.
        while path != pathlib.PurePath('/'):
            size[path.parent] = size.get(path.parent, 0) + total
            path = path.parent

    return size
      
total(build_fs(parse(example)))

{PurePosixPath('/'): 48381165,
 PurePosixPath('/a'): 94853,
 PurePosixPath('/a/e'): 584,
 PurePosixPath('/d'): 24933642}

Sum directories of size at most 100000.

In [5]:
sum(
    size 
    for size in total(build_fs(parse(example))).values() 
    if size <= 100000
)

95437

# Part 1

Sum directories of size at most 100000 in input.

In [6]:
sum(
    size 
    for size in total(build_fs(parse(open('day-7-input.txt').read()))).values() 
    if size <= 100000
)

1297159

# Part 2

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update.

In [7]:
available = 70000000
required = 30000000

totals = total(build_fs(parse(example)))
used = available - totals[pathlib.PurePath('/')]

min(
    size 
    for size in totals.values()
    if used + size >= required
)

24933642

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update.

In [8]:
totals = total(build_fs(parse(open('day-7-input.txt').read())))
used = available - totals[pathlib.PurePath('/')]

min(
    size 
    for size in totals.values()
    if used + size >= required
)

3866390